# 줄리아를 생각하다(Think julia)

## Chapter 18. 하위 유형화(subtyping)

* 매서드의 시그니처에 허용되는 자료형이 무엇인지 명시

* 트럼프 카드와 카드의 덱, 카드 조합(패)을 나타내는 자료형을 이용해 하위 자료형 정의를 살펴볼 예정

### 18.1 카드

* 카드를 나타내는 객체 정의 : rank(끗수)와 suit(무늬) 이용

* rank(끗수)와 suit(무늬)는 **부호화** (encode)한 숫자를 이용 (높은 무늬는 높은 숫자에 대응)

    - ♠ ↦ 4
    - ♥ ↦ 3
    - ♦ ↦ 2
    - ♣ ↦ 1


* 카드 구조체 정의

In [1]:
struct Card
    suit :: Int64
    rank :: Int64
    function Card(suit::Int64, rank::Int64)
        @assert(1 ≤ suit ≤ 4, "suit is not between 1 and 4")
        @assert(1 ≤ rank ≤ 13, "rank is not between 1 and 13")
        new(suit, rank)
    end
end 

* 카드 한 장을 만들려면, 원하는 무늬와 끗수를 인수로 생성자 Card를 호출함

In [2]:
queen_of_diamonds = Card(2, 12)

Card(2, 12)

### 18.2 전역 변수

* Card 객체를 알아보기 쉽게 출력하려면, 숫자 부호로 끗수와 무늬를 가져올 수 있는 사상이 필요함

* 불변 전역변수를 선언하는 const 명령어를 이용하여 할당

    - ```const``` is used to declare global variables whose values will not change. In almost all code (and particularly performance sensitive code) global variables should be declared constant in this way.

In [3]:
const suit_names = ["♣", "◆", "♥", "♠"]
const rank_names = ["A", "2", "3" , "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]

13-element Vector{String}:
 "A"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 "9"
 "10"
 "J"
 "Q"
 "K"

* show 메서드 구현

* Card 구조체의 인스턴스 호출 시 화면 출력 가능

In [4]:
function Base.show(io::IO, card::Card)
    print(io, rank_names[card.rank], suit_names[card.suit])
end

In [5]:
Card(3, 11)

J♥

### 18.3 카드 비교

* 내장 자료형의 경우 값을 비교해서 어떤 것이크고 작은지, 또는 같은 지를 알 수 있는 >, <, == 등의 관계 연산자가 제공

* 사용자 정의 자료형의 경우 < 메서드를 정의함으로써 내장된 연산자들이 제대로 동작하도록 만들 수 있음

* 여기서는 단순하게 무늬가 더 중요하다고 임의로 설정하고 무늬가 높으면 순서가 높다고 설정

In [6]:
import Base.isless

function isless(c1::Card, c2::Card)
    (c1.suit, c1.rank) < (c2.suit, c2.rank)
end

isless (generic function with 44 methods)

### 18.4 유닛 테스트

* **유닛 테스트(unit test)** : 작성한 코드의 실행 결과를 예상 결과와 비교하는 것

    * @test는 뒤에 나오는 표현식이 참일 경우 "Test Passed"를 반환

    * 거짓이면 "Test Failed"

    * 표현식을 평가할 수 없는 경우에는 "Error Result"를 반환

In [11]:
using Test

In [12]:
@test Card(1, 4) < Card(2, 4)

Test Passed

In [13]:
@test Card(1, 3) < Card(1, 4)

Test Passed

### 18.5 덱(Decks)

* **덱** (한 묶음의 카드, 52장)을 표현하는 구조체 Deck 정의

    * 덱은 카드의 모음이기 때문에 각각의 덱이 카드의 배열을 속성으로 가지고 있는 것이 자연스러움


* 구조체 Deck: 생성자는 cards 속성을 만든 후, 52장으로 이루어진 표준적인 카드 한 벌을 생성하여 cards 속성에 넣음

In [8]:
struct Deck
    cards :: Array{Card, 1}
end

function Deck()
    deck = Deck(Card[])
    for suit in 1:4
        for rank in 1:13
            push!(deck.cards, Card(suit, rank))
        end
    end
    deck
end

Deck

* Deck의 show 메서드 설정

In [9]:
function Base.show(io::IO, deck::Deck)
    for card in deck.cards
        print(io, card, " ")
    end
    println()
end

* 결과 확인

In [10]:
Deck()

A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A◆ 2◆ 3◆ 4◆ 5◆ 6◆ 7◆ 8◆ 9◆ 10◆ J◆ Q◆ K◆ A♥ 2♥ 3♥ 4♥ 5♥ 6♥ 7♥ 8♥ 9♥ 10♥ J♥ Q♥ K♥ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ K♠ 

### 18.6 더하기, 빼기, 섞기, 정렬하기

* 카드를 다루려면 Deck에서 Card를 빼서 반환하는 함수 등이 필요

* 다른 메서드를 활용해서 처리하는 메서드를 **겉판(베니어)** veneer 라고 부름

* pop!과 shuffle! 함수의 메서드에 Deck과 Card의 구조체에 관한 메서드를 추가함

In [14]:
# 카드 빼기

function Base.pop!(deck::Deck)
    pop!(deck.cards, card)
end

In [15]:
# 카드 넣기

function Base.push!(deck::Deck, card::Card)
    push!(deck.cards, card)
    deck
end

In [16]:
# 카드 섞기

using Random

function Random.shuffle!(deck::Deck)
    shuffle!(deck.cards)
    deck
end

### 18.7 추상 자료형과 하위 유형화


**하위 유형화(서브 타이핑) subtyping**

* **상위 추상 자료형(abstract type)** 을 정의함으로써 Deck과 Hand의 공통 속성을 가지는 **구체 자료형(concrete type)** 을 묶어서 처리 가능

* **상위 추상 자료형(abstract type) 정의** 

    - 새로운 추상 자료형은 abstract type 예약어로 생성

    - 자료형을 만들때 상위 자료형을 지정하고자 한다면 자료형 이름 뒤에 ```<:```을 쓰고, 상위 자료형으로 지정할 기존 자료형의 이름을 붙임

    - 상위 자료형(abstract type)이 주어지지 않으면 기본적으로 Any가 상위 자료형이 됨

        * 모든 객체는 Any의 인스턴스이고, 모든 자료형은 Any의 하위 자료형

- **(예시)** Deck과 Hand의 공통 속성을 가지는 추상 자료형을 CardSet 생성

    * 하위 자료형으로 플레이어가 손에 들고 있는 카드를 나타내는 자료형 **hand** 생성

        * deck과 유사하게 card의 모음으로 이루어져 있으며, card를 더하고 빼는 메서드가 필요함
    
        * deck에는 해당되지 않지만 hand에서는 필요한 연산들 고려 필요
    
            + 두 개의 패를 비교하여 승부 확인 연산
        
            + 컨트랙트 브릿지와 같은 게임이라면 입찰을 위한 패의 점수 계산과 같은 연산


In [27]:
abstract type CardSet end

In [31]:
struct Deck2 <: CardSet
    cards::Array{Card, 1}
end 

function Deck2()
    deck = Deck2(Card[])
    for suit in 1:4
        for rank in 1:13
            push!(deck.cards, Card(suit, rank))
        end
    end
    deck
end

Deck2

In [32]:
deck = Deck2();

deck isa CardSet

true

* Hand 역시 CardSet의 한 종류

* Hand의 생성자는 빈 배열 할당 (초기화)

In [33]:
struct Hand <: CardSet
    cards::Array{Card, 1}
    label::String
end

function Hand(label::String="")
    Hand(Card[], label)
end

Hand

In [34]:
hand = Hand("new hand");

println(hand)
hand isa CardSet

Hand(Card[], "new hand")


true

### 18.8 추상 자료형과 함수

In [40]:
function Base.show(io::IO, cs::CardSet)
    for card in cs.cards
        print(io, card, " ")
    end
end

function Base.pop!(cs::CardSet)
    pop!(cs.cards)
end

function Base.push!(cs::CardSet, card::Card)
    push!(cs.cards, card)
    nothing
end

using Random

function Random.shuffle!(deck::CardSet)
    shuffle!(deck.cards)
    deck
end

In [41]:
deck = Deck2()

A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A◆ 2◆ 3◆ 4◆ 5◆ 6◆ 7◆ 8◆ 9◆ 10◆ J◆ Q◆ K◆ A♥ 2♥ 3♥ 4♥ 5♥ 6♥ 7♥ 8♥ 9♥ 10♥ J♥ Q♥ K♥ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ K♠ 

In [42]:
shuffle!(deck)

6♠ 9♥ 10♥ 7♥ J◆ 5◆ 9◆ 4♠ Q♠ 10◆ 8◆ J♥ A◆ 5♣ 7◆ 5♠ A♥ 9♣ 7♠ A♠ K♥ 2♣ Q♥ K♠ Q♣ 9♠ A♣ 8♣ 7♣ 4◆ 8♠ J♠ 2◆ 2♥ 4♥ 2♠ 3♣ 6♥ 8♥ 10♣ 6♣ 4♣ 3♠ Q◆ 3♥ K♣ 10♠ 5♥ 3◆ 6◆ J♣ K◆ 

In [43]:
card = pop!(deck)

K◆

In [44]:
push!(hand, card)

In [45]:
hand

K◆ 

* 자연스럽게 카드 이동을 move!라는 함수로 감쌀 수도 있음

In [46]:
function move!(cs1::CardSet, cs2::CardSet, n::Int)
    @assert 1 ≤ n ≤ length(cs1.cards)
    for i in 1:n
        card = pop!(cs1)
        push!(cs2, card)
    end
    nothing
end

move! (generic function with 1 method)

* move! 함수는 두 개의 CardSet 객체와 이동시킬 카드의 수를 인수로 받음

* 함수는 CardSet 양쪽을 모두 수정하고, nothing을 반환함

* 어떤 게임에서는 카드가 이 패에서 다른 패로 넘어가거나, 패에서 덱으로 돌아감 -> 이 경우 move! 함수 사용 가능

In [47]:
move!(deck, hand, 3)

In [48]:
hand

K◆ J♣ 6◆ 3◆ 

### 18.9 자료형 도식(Type diagram)

**자료형 도식(Type diagram)** : 자료형 간 관계를 보여주는 도식

![image.png](capture_and_drawing/Think_julia_drawing2.png)

### 18.10 디버깅

* 하위 유형화는 어떤 객체를 인수로 함수를 호출했을 때, 정확히 어떤 메서드가 실행되는 지 알기 어렵기 때문에 디버깅을 더 어렵게 만들 수 있음

    - 어떤 하위 자료형에 대해 정의된 메서드가 있다면 그것이 대신 실행될 것이기 때문


* 메서드의 앞부분에 print문을 추가하여 프로그램 실행 시 사용되는 자료형을 출력하거나, @which 매크로를 사용하여 해결 할 수 있음

    - 아래 예시에 따르면 sort! 메서드가 자료형 Hand의 객체를 인수로 받는 버전임을 알려줌

In [50]:
function Base.sort!(hand::Hand)
    sort!(hand.cards)
end

In [51]:
@which sort!(hand)

sort!(hand::Hand) in Main at In[50]:1

* **리스코프 치환 원칙(Liskov substitution principle)**

    -  객체 지향 프로그래밍 원칙이다. 컴퓨터 프로그램에서 자료형 S가 자료형 T의 하위형이라면 필요한 프로그램의 속성(정확성, 수행하는 업무 등)의 변경 없이 자료형 T의 객체를 자료형 S의 객체로 교체(치환)할 수 있어야 한다는 원칙이다. 

    * (참조) https://ko.wikipedia.org/wiki/%EB%A6%AC%EC%8A%A4%EC%BD%94%ED%94%84_%EC%B9%98%ED%99%98_%EC%9B%90%EC%B9%99
    
    * CardSet와 같이 상위 자료형에 대해서 동작하도록 만들어진 함수가 Deck과 Hand 같은 하위 자료형에도 잘 동작해야 한다는 원칙


* supertype 함수는 어떤 자료형의 바로 위 상위 자료형을 찾는데 사용됨

In [53]:
supertype(Deck)

Any

### 18.11 자료 캡슐화(Data Encapsulation)

* **자료형 중심 설계(type-oriented design)** 필요한 객체와 그것을 표현할 수 있는 구조체 정의

* **자료 캡슐화(data encapsulation)** 캡슐화와 일반화를 통해 함수의 인터페이스 발견

* **예시)** **마르코프분석**

    * 여러 함수에서 읽고 쓰는 전역 변수 suffix와 prefix 객체 생성
    
    * 여러 개의 분석을 시행하려면, 각 분석의 상태를객체 내부로 숨김(캡슐화)
    
    * 함수를 메서드로 변환(password 함수)

In [54]:
suffixes = Dict()
prefix = []

struct Markov
    order::Int64
    suffixes::Dict{Tuple{string, Vararg{String}}, Array{String, 1}}
    prefix::Array{String, 1}
end

function Markov(order::Int64 = 2)
    new(order, Dict{Tuple{String, Vararg{String}}, Array{String, 1}}(),
        Array{String, 1}())
end

Markov

In [55]:
function processword(markov::Markov, word::String)
    if length(markov.prefix) < markov.order
        push!(markov.prefix, word)
        return
    end
    get!(markov.suffixes, (markov.prefix...,), Array{String, 1,}())
    push!(markov.suffixes[(markov.prefix...,)], word)
    popfirst!(markov.prefix)
    push!(markov.prefix, word)
end

processword (generic function with 1 method)

* **(정리)** 캡슐화 설계 방식 

    - 1) (필요하다면) 전역 변수를 읽고 쓰는 함수 작성
    
    - 2) 프로그램이 동작하면, 전역 변수와 그 전역 변수에 접근하는 함수의 연관 관계를 살펴 봄
    
    - 3) 관련된 변수를 구조체의 필드로 캡슐화 함
    
    - 4) 관련된 함수를 새로운 자료형을 인수로 받는 메서드로 바꿔 줌

### 18.12 용어집

* **부호화** (encode)

    - 어떤 값의 집합을 다른 값의 집합으로 대응시켜 표현을 바꾸는 것
```
```
* **유닛 테스트** (unit test)

    - 코드의 정확성을 시험해보는 표준적인 방법
```
```
* **겉판** (vaneer)

    - 어떤 연산없이 어떤 함수에 대한 다른 인터페이스를 제공하는 메서드나 함수
```
```
* **구체 자료형** (concrete type)

    - 객체를 만들 수 있는 자료형
```
```
* **구체 자료형** (concrete type)

    - 객체를 만들 수 있는 자료형
```
```
* **추상 자료형** (subtract type)

    - 어떤 자료형들의 상위 자료형이 될 수 있는 자료형. 직접 객체 생성 안됨
```
```
* **하위 유형화** (subtyping)

    - 관련 있는 자료형을 위계적으로 정의하는 것
```
```
* **상위 자료형** (supertype)

    - 여러 자료형들의 공통적인 성질을 가지는 추상 자료형
```
```
* **하위 자료형** (subtype)

    - 어떤 추상 자료형을 상위 자료형으로 가지는 자료형
```
```
* **자료형 도식** (type diagram)

    - 프로그램의 자료형과 그들의 관계를 표시한 도식
```
```
* **has-a 관계** (has-a relationship)

    - 한 자료형의 인스턴스가 다른 자료형이 인스턴스를 참조하는 관계
```
```
* **is-a 관계** (is-a relationship)

    - 하위 자료형과 상위 자료형의 관계
```
```
* **의존성** (dependency)

    - 한 자료형의 인스턴스가 다른 자료형의 인스턴스를 사용하지만, 필드로 저장하지 않는 관계
```
```
* **다중도** (multiplicity)

    - 자료형 도식에서 has-a 관계를 표현할 때, 다른 자료형의 인스턴스를 얼마나 많이 참조하는지 표시하는 방법
```
```
* **자료캡슐화** (data encapsulation)

    - 전역 변수를 이용한 프로토타입을 만든 후, 이들 전역 변수를 필드로 캡슐화한 최종 버전으로 바꿔나가는 개발 계획